In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, SpatialDropout1D
from keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.utils import pad_sequences
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 255)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [4]:
import numpy as np
import pandas as pd


In [7]:

url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
!wget {url}

archive_name = url.split('/')[-1]
!tar -xf {archive_name}

print(f"IMDB dataset downloaded and extracted from {archive_name}")

dataset_dir = './aclImdb'


print("\nDataset directory structure:")
!ls -R {dataset_dir}

Streaming output truncated to the last 5000 lines.
12999_0.txt  20500_0.txt  2800_0.txt   3550_0.txt   43001_0.txt  5502_0.txt
13000_0.txt  2050_0.txt   28001_0.txt  35501_0.txt  43002_0.txt  5503_0.txt
1300_0.txt   20501_0.txt  28002_0.txt  35502_0.txt  43003_0.txt  5504_0.txt
13001_0.txt  20502_0.txt  28003_0.txt  35503_0.txt  43004_0.txt  5505_0.txt
13002_0.txt  20503_0.txt  28004_0.txt  35504_0.txt  43005_0.txt  5506_0.txt
13003_0.txt  20504_0.txt  28005_0.txt  35505_0.txt  43006_0.txt  5507_0.txt
13004_0.txt  20505_0.txt  28006_0.txt  35506_0.txt  43007_0.txt  5508_0.txt
13005_0.txt  20506_0.txt  28007_0.txt  35507_0.txt  43008_0.txt  5509_0.txt
13006_0.txt  20507_0.txt  28008_0.txt  35508_0.txt  43009_0.txt  55_0.txt
13007_0.txt  20508_0.txt  28009_0.txt  35509_0.txt  430_0.txt	 5510_0.txt
13008_0.txt  20509_0.txt  280_0.txt    355_0.txt    43010_0.txt  551_0.txt
13009_0.txt  205_0.txt	  28010_0.txt  35510_0.txt  4301_0.txt	 5511_0.txt
130_0.txt    20510_0.txt  2801_0.txt   3551_

In [9]:
import pandas as pd
import os

def load_imdb_data(path):
    data = {'text': [], 'sentiment': []}
    for sentiment_type in ['neg', 'pos']:
        sentiment_path = os.path.join(path, sentiment_type)

        if not os.path.exists(sentiment_path):
            print(f"Warning: Directory not found: {sentiment_path}")
            continue
        for filename in os.listdir(sentiment_path):

            if filename.endswith('.txt'):
                with open(os.path.join(sentiment_path, filename), 'r', encoding='utf-8') as f:
                    data['text'].append(f.read())
                    data['sentiment'].append(1 if sentiment_type == 'pos' else 0)
    return pd.DataFrame(data)

dataset_dir = './aclImdb'


train_path = os.path.join(dataset_dir, 'train')
train_df = load_imdb_data(train_path)
print(f"\nTraining data loaded, shape: {train_df.shape}")
display(train_df.head())


test_path = os.path.join(dataset_dir, 'test')
test_df = load_imdb_data(test_path)
print(f"\nTest data loaded, shape: {test_df.shape}")
display(test_df.head())


Training data loaded, shape: (25000, 2)


,text,sentiment
0,"Making the film as dark and visually fuzzy as possible in order to cover up the budget deficiencies is an often-used strategy in low-budget horror films, but this one takes it too far. It is SO poorly lit and murky (and it takes place almost entirely ...",0
1,"A trio of low-life criminals, led by Matt Dillon, botches a late-night burglary. They flee but quickly cross paths with the police who just happen to be in hot pursuit of a terrorist. Of course the police mistake the burglar gang for the terrorist, th...",0
2,There is a reason why certain films go straight to video and of course the obvious reason is that if its too naughty for theater audience then release straight to video. Of course it really wouldn't be fair to the films that are good and yet they are ...,0
3,Would someone tell shaq to stick to what he is good at basketball. This movie was not even entertaining on a stupid level. In this movie shaq plays a genie who lives in a boom box is that not orginal a genie in a boom box instead of a lamp. He is supp...,0
4,"The past creeps up on a rehab-addict when he reconnects with his ill brother and a former girlfriend after what he hopes was his last stint in detox. ""Life's dramas"", presented here in the most simplistic way imaginable (not even the writing has any b...",0



Test data loaded, shape: (25000, 2)


,text,sentiment
0,1st watched 4/30/2009 - 4 out of 10 (Dir-John Waters): Corny Waters-like comedy musical with some funny scenes and good parts but it didn't make a whole worthwhile experience. John Waters directed this music-filled spoof of the fifties scene with John...,0
1,"***SPOILERS*** ***SPOILERS*** HERE ON EARTH / (2000) 1/2* (out of four)<br /><br />Mark Piznarski's ""Here on Earth"" holds the record for a movie containing the most recycled material in 96 minutes. Literally every contrivance, cliché, and familiar plo...",0
2,"I give this piece of Hollywood trash 1 out 10! Seriously! I mean, I like comedy as much as the next guy. I also can take just plain stupid comedy and actually sit back and laugh with it. But this film had nothing to laugh at OR with.<br /><br />I like...",0
3,"I have seen both the MST3K version and the uncut version. I rather enjoyed it. Either way, it wasn't that bad of a movie. Sure it moved a bit slow at times. I liked it.<br /><br />As far as MST3K goes, they only did the movies they could get the right...",0
4,"I've been on a bad run of films. This is a clinker about an arson plot and a psychopath. Tom Skerrit, whom I really enjoy, was pretty young here. He is a builder with a passion, but he has a partner whose profit motive includes over-insuring and burni...",0


In [11]:
train_df['text'][7]

"It's the early 80s. There's a group of suspiciously old-looking teens. And there's a maniac stalking around. Yes, this is slasherville.<br /><br />This movie is called Pranks. Why is it called Pranks? I haven't the faintest idea. Unless your idea of a great prank is to repeatedly hit someone's dinner with a baseball bat - on balance, not a great prank; in fact quite a rubbish prank if truth be told. But there you go.<br /><br />The film itself concerns a group of teenagers who are tasked with cleaning out a decommissioned dormitory. They become aware that a psychopath is on the loose. To combat this development, they split up and wander about in the dark. It ends in tears for most of them.<br /><br />Pranks is a badly made slasher movie. The DVD release I viewed was the Vipco one. It appears to be cut of a fair bit of violence. This makes the DVD even more pointless because, let's face it, a slasher movie shorn of violence is a waste of time. For slasher-film and video nasty completis

In [13]:
train_df['sentiment'].value_counts()

,count
sentiment,
0,12500
1,12500


In [14]:
train_df.isnull().sum()

,0
text,0
sentiment,0


In [15]:
train_df.duplicated().sum()

np.int64(96)

In [16]:
train_df.drop_duplicates(inplace=True)
